In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime, timedelta
from control.modules.thrusters import (
    ThrustersControl,
    ThrustersParameters,
)
from input_output.base import Stamped
from input_output.definitions.simulation import (
    Boundary,
    Pcs,
    TemperatureBoundary,
    Thruster,
)
from input_output.modules.thrusters import (

    ThrustersSensorValues,
    ThrustersSimulationInputs,
    ThrustersSimulationOutputs,
)
from orchestration.collector import PolarsCollector

from orchestration.executor import SimulationExecutor

from simulation.fmu import Fmu
from simulation.io_mapping import IoMapping
from simulation.models.fmu_paths import thrusters_path



collector = PolarsCollector()

executor = SimulationExecutor(
    IoMapping(Fmu(thrusters_path), ThrustersSensorValues, ThrustersSimulationOutputs),
    ThrustersSimulationInputs(
        thrusters_aft=Thruster(
            heat_flow=Stamped.stamp(9000), active=Stamped.stamp(True)
        ),
        thrusters_fwd=Thruster(
            heat_flow=Stamped.stamp(4300), active=Stamped.stamp(True)
        ),
        thrusters_seawater_supply=Boundary(
            temperature=Stamped.stamp(32), flow=Stamped.stamp(64)
        ),
        thrusters_module_supply=TemperatureBoundary(temperature=Stamped.stamp(50)),
        thrusters_pcs=Pcs(mode=Stamped.stamp("propulsion")),
    ),
    datetime.now(),
    timedelta(seconds=1),
)
control = ThrustersControl(ThrustersParameters())


control_values = control.initial(executor.time()).values

#stabilize
for i in range(90):
    result = await executor.tick(control_values)
    collector.collect(result.raw, result.timestamp)

control.to_cooling()  # type: ignore
# set valves and stabilize 10s
for i in range(100):
    control_values = control.control(result.sensor_values, executor.time()).values
    result = await executor.tick(control_values)
    collector.collect(result.raw, result.timestamp)

#assert stable flows
for i in range(60):
    control_values = control.control(result.sensor_values, executor.time()).values
    result = await executor.tick(control_values)
    collector.collect(result.raw, result.timestamp)

#aft thruster off
executor._boundaries.thrusters_aft.active = Stamped.stamp(False)
executor._boundaries.thrusters_aft.heat_flow = Stamped.stamp(0)

# set valves and stabilize 30s
for i in range(120):
    control_values = control.control(result.sensor_values, executor.time()).values
    result = await executor.tick(control_values)
    collector.collect(result.raw, result.timestamp)

#assert stable flows
for i in range(60):
    control_values = control.control(result.sensor_values, executor.time()).values
    result = await executor.tick(control_values)
    collector.collect(result.raw, result.timestamp)

#shut down fwd thruster
executor._boundaries.thrusters_fwd.active = Stamped.stamp(False)
executor._boundaries.thrusters_fwd.heat_flow = Stamped.stamp(0)

# set valves and stabilize 30s
for i in range(120):
    control_values = control.control(result.sensor_values, executor.time()).values
    result = await executor.tick(control_values)
    collector.collect(result.raw, result.timestamp)

#assert zero flows
for i in range(60):
    control_values = control.control(result.sensor_values, executor.time()).values
    result = await executor.tick(control_values)
    collector.collect(result.raw, result.timestamp)


[WARNING] Warning: The following was detected at time: 4.686242511022535E-26
[WARNING]   Thruster_Module_FMU.thrusterModule.flowJoin_mixing_return: Flow splitRatio is far above 1. But it must be between 0 and 1. (is 1.00983)
  Failed condition: thrusterModule.flowJoin_mixing_return.splitRatio <= 1.001

[WARNING] Warning: The following was detected at time: 2.010123200637635E-23
[WARNING]   Thruster_Module_FMU.thrusterModule.flowJoin_thruster_out: Flow splitRatio is negative and far below 0. But it must be between 0 and 1. (is -3.30264e+15)
  Failed condition: thrusterModule.flowJoin_thruster_out.splitRatio >= -0.001

[WARNING] Warning: The following was detected at time: 1.532796717500692E-10
[WARNING]   Thruster_Module_FMU.thrusterModule.flowJoin_AFT: Flow splitRatio is far above 1. But it must be between 0 and 1. (is 1.001)
  Failed condition: thrusterModule.flowJoin_AFT.splitRatio <= 1.001

[WARNING] Warning: The following was detected at time: 6.535411437471817E-08
[WARNING]   Thru

In [13]:
collector.result().write_csv('thrusters_test_data.csv')